In [2]:
import cv2
import numpy as np
import dlib
from imutils import face_utils
import time

# Función para calcular IoU (Intersection over Union) entre dos rectángulos
def calcular_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    
    if union_area == 0:
        return 0
    
    return inter_area / union_area

# Función para filtrar detecciones solapadas
def filtrar_solapamientos(detecciones, umbral_iou=0.3):
    detecciones_filtradas = []
    
    for rect in detecciones:
        agregar = True
        for f_rect in detecciones_filtradas:
            if calcular_iou(rect, f_rect) > umbral_iou:
                agregar = False
                break
        if agregar:
            detecciones_filtradas.append(rect)
    return detecciones_filtradas

# Cargar modelo entrenado
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('modeloLBPHFace.xml')

# Cargar clasificadores
face_frontal = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_alt = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_perfil = cv2.CascadeClassifier('haarcascade_profileface.xml')

# Detector y predictor dlib
detector_dlib = dlib.get_frontal_face_detector()
predictor_dlib = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

usuarios = ["Marcelo", "Samuel", "Yoana"]

umbral_reconocimiento = 70
tiempo_estatico = 7.0
tiempo_inicial_parpadeo = time.time()

cap = cv2.VideoCapture(0)

rostro_anterior = None
contador_estatico = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = frame.copy()

    detected_faces = []

    faces_frontal = face_frontal.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(40, 40))
    faces_alt = face_alt.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(40, 40))
    faces_perfil = face_perfil.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))
    faces_perfil_flip = face_perfil.detectMultiScale(cv2.flip(gray, 1), scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    for face_list in [faces_frontal, faces_alt, faces_perfil]:
        if len(face_list) > 0:
            detected_faces.extend(face_list)

    for (x, y, w, h) in faces_perfil_flip:
        x_corrected = gray.shape[1] - x - w
        detected_faces.append((x_corrected, y, w, h))

    # Filtrar solapamientos para evitar detecciones dobles cerca una de otra
    detected_faces = filtrar_solapamientos(detected_faces, umbral_iou=0.3)

    texto_mostrar = []
    color_mostrar = []

    for (x, y, w, h) in detected_faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (128, 0, 255), 2)
        rostro = auxFrame[y:y + h, x:x + w]
        rostro_gray = cv2.cvtColor(cv2.resize(rostro, (150, 150)), cv2.COLOR_BGR2GRAY)

        # Verificar imagen estática
        if rostro_anterior is not None:
            diferencia = np.sum(np.abs(rostro_anterior - rostro_gray))
            if diferencia < 600:
                contador_estatico += 1
            else:
                contador_estatico = 0

        tiempo_actual_parpadeo = time.time()
        rects = detector_dlib(gray, 0)

        texto = "No detectado"
        color = (0, 0, 255)

        for rect in rects:
            shape = predictor_dlib(gray, rect)
            shape = face_utils.shape_to_np(shape)

            for (sx, sy) in shape:
                cv2.circle(frame, (sx, sy), 2, (0, 255, 0), -1)

            ojo_izquierdo = shape[36:42]
            ojo_derecho = shape[42:48]
            ratio_izquierdo = np.linalg.norm(ojo_izquierdo[1] - ojo_izquierdo[5]) / np.linalg.norm(ojo_izquierdo[0] - ojo_izquierdo[3])
            ratio_derecho = np.linalg.norm(ojo_derecho[1] - ojo_derecho[5]) / np.linalg.norm(ojo_derecho[0] - ojo_derecho[3])
            ratio_parpadeo = (ratio_izquierdo + ratio_derecho) / 2

            if ratio_parpadeo < 0.18:
                tiempo_inicial_parpadeo = time.time()
                id_usuario, confianza = face_recognizer.predict(rostro_gray)

                if confianza < umbral_reconocimiento and id_usuario < len(usuarios):
                    texto = f"{usuarios[id_usuario]} - Confianza: {confianza:.2f}"
                    color = (0, 255, 0)
                else:
                    texto = "Desconocido"
                    color = (0, 0, 255)
            else:
                if tiempo_actual_parpadeo - tiempo_inicial_parpadeo >= tiempo_estatico:
                    texto = "Imagen estática detectada"
                    color = (0, 128, 255)
                else:
                    id_usuario, confianza = face_recognizer.predict(rostro_gray)

                    if confianza < umbral_reconocimiento and id_usuario < len(usuarios):
                        texto = f"{usuarios[id_usuario]} - Confianza: {confianza:.2f}"
                        color = (0, 255, 0)
                    else:
                        texto = "Desconocido"
                        color = (0, 0, 255)

        rostro_anterior = rostro_gray.copy()

        texto_mostrar.append((texto, (x, max(y - 15, 30)), color))

    # Mostrar solo el texto final para cada detección, para evitar solapamiento de textos
    for texto, pos, color in texto_mostrar:
        cv2.putText(frame, texto, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)

    cv2.imshow('Reconocimiento Facial', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
